In [2]:
import pandas as pd

In [17]:
df_test = pd.read_csv("data/preprocessed_data/t2_test.csv")
print(df_test)

       year  month  day  mean_temperature  total_precipitation  isHoliday   
0      2022      4   13               9.2                  8.2      False  \
1      2022      4   13               9.2                  8.2      False   
2      2022      4   13               9.2                  8.2      False   
3      2022      4   13               9.2                  8.2      False   
4      2022      4   13               9.2                  8.2      False   
...     ...    ...  ...               ...                  ...        ...   
39815  2022     10   31               9.5                  0.0      False   
39816  2022     10   31               9.5                  0.0      False   
39817  2022     10   31               9.5                  0.0      False   
39818  2022     10   31               9.5                  0.0      False   
39819  2022     10   31               9.5                  0.0      False   

       isWeekend  stations_cluster  rides_count  
0          False         